# ML - Problem Set 2: Predicting Poverty

El objetivo principal es construir un modelo predictivo de la pobreza de los hogares. Nótese que un hogar se clasifica como

\begin{equation}
    Poor = I \left( Inc < Pl \right)
\end{equation}

donde $I$ es una función indicadora que toma uno si el ingreso familiar está por debajo de una determinada línea de pobreza.

## Paso 1. Procesamiento de datos

In [ ]:
# Importar librerias

import pandas as pd
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importar datos

test_hogares = pd.read_csv('test_hogares.csv')
train_hogares = pd.read_csv('train_hogares.csv')
test_personas = pd.read_csv('test_personas.csv')
train_personas = pd.read_csv('train_personas.csv')

In [3]:
# Limpieza de datos

train_personas['P6210s1'] = train_personas['P6210s1'].clip(lower=0, upper=15) #eliminar valores extremos
test_personas['P6210s1'] = test_personas['P6210s1'].clip(lower=0, upper=15) #eliminar valores extremos

train_personas['P7510s5'] = train_personas['P7510s5'].clip(lower=1, upper=2) #eliminar valores extremos
test_personas['P7510s5'] = test_personas['P7510s5'].clip(lower=1, upper=2) #eliminar valores extremos

train_personas['Des'] = train_personas['Des'].fillna(0) #reemplazar nan por 0
train_personas['Ina'] = train_personas['Ina'].fillna(0) #reemplazar nan por 0
test_personas['Des'] = test_personas['Des'].fillna(0) #reemplazar nan por 0
test_personas['Ina'] = test_personas['Ina'].fillna(0) #reemplazar nan por 0

valores_unicos = sorted(train_hogares['Pobre'].unique())
print(valores_unicos)

[0, 1]


In [4]:
# Imputar valores faltantes

imputador_moda = SimpleImputer(strategy='most_frequent')  # Para variables categóricas
imputador_mediana = SimpleImputer(strategy='median')  # Para variables numéricas

# Función para imputar datos faltantes con SimpleImputer
def imputar_datos_simple(df):
    # Separar columnas categóricas y numéricas
    columnas_categoricas = df.select_dtypes(exclude=['number']).columns
    columnas_numericas = df.select_dtypes(include=['number']).columns

    # Imputar categóricas con la moda
    if not columnas_categoricas.empty:
        df[columnas_categoricas] = imputador_moda.fit_transform(df[columnas_categoricas])

    # Imputar numéricas con la mediana
    if not columnas_numericas.empty:
        df[columnas_numericas] = imputador_mediana.fit_transform(df[columnas_numericas])

    return df

# Cargar los datos desde archivos CSV (asegúrate de que las rutas sean correctas)
test_hogares = pd.read_csv('test_hogares.csv')
train_hogares = pd.read_csv('train_hogares.csv')
test_personas = pd.read_csv('test_personas.csv')
train_personas = pd.read_csv('train_personas.csv')

# Aplicar la imputación simple a cada DataFrame
test_hogares = imputar_datos_simple(test_hogares)
train_hogares = imputar_datos_simple(train_hogares)
test_personas = imputar_datos_simple(test_personas)
train_personas = imputar_datos_simple(train_personas)

# Verificar valores faltantes después de imputación
print("Valores faltantes después de imputación:")
print("test_hogares:", test_hogares.isnull().sum().sum())
print("train_hogares:", train_hogares.isnull().sum().sum())
print("test_personas:", test_personas.isnull().sum().sum())
print("train_personas:", train_personas.isnull().sum().sum())

Valores faltantes después de imputación:
test_hogares: 0
train_hogares: 0
test_personas: 0
train_personas: 0


In [5]:
print(test_hogares.columns.tolist())
print(train_hogares.columns.tolist())
print(test_personas.columns.tolist())
print(train_personas.columns.tolist())

['id', 'Clase', 'Dominio', 'P5000', 'P5010', 'P5090', 'P5100', 'P5130', 'P5140', 'Nper', 'Npersug', 'Li', 'Lp', 'Fex_c', 'Depto', 'Fex_dpto']
['id', 'Clase', 'Dominio', 'P5000', 'P5010', 'P5090', 'P5100', 'P5130', 'P5140', 'Nper', 'Npersug', 'Ingtotug', 'Ingtotugarr', 'Ingpcug', 'Li', 'Lp', 'Pobre', 'Indigente', 'Npobres', 'Nindigentes', 'Fex_c', 'Depto', 'Fex_dpto']
['id', 'Orden', 'Clase', 'Dominio', 'P6020', 'P6040', 'P6050', 'P6090', 'P6100', 'P6210', 'P6210s1', 'P6240', 'Oficio', 'P6426', 'P6430', 'P6510', 'P6545', 'P6580', 'P6585s1', 'P6585s2', 'P6585s3', 'P6585s4', 'P6590', 'P6600', 'P6610', 'P6620', 'P6630s1', 'P6630s2', 'P6630s3', 'P6630s4', 'P6630s6', 'P6800', 'P6870', 'P6920', 'P7040', 'P7045', 'P7050', 'P7090', 'P7110', 'P7120', 'P7150', 'P7160', 'P7310', 'P7350', 'P7422', 'P7472', 'P7495', 'P7500s2', 'P7500s3', 'P7505', 'P7510s1', 'P7510s2', 'P7510s3', 'P7510s5', 'P7510s6', 'P7510s7', 'Pet', 'Oc', 'Des', 'Ina', 'Fex_c', 'Depto', 'Fex_dpto']
['id', 'Orden', 'Clase', 'Domini

## Fusionar individuos con hogares

Se crean las siguientes variables:
- Promedio_Educacion: se espera que a mayor nivel educativo del hogar, mayor sera el ingreso

- hacinamiento: se supone que a mayor hacinamiento, mas probabilidad tiene el hogar de ser pobre

- capacitacion_financiera: se supone que los hogares pobres no tienen capaticacion financiera

- jefe_sex: es posible que los hogares con jefas de hogares mujeres tengan menos ingresos que aquellso donde el jefe de hogar es hombre

In [6]:
# Crear una nueva variable que sea promedio educativo por hogar considerando individuos mayores de 18 años

# Filtrar personas mayores de 18 años en el dataset de entrenamiento
train_personas_mayores = train_personas[train_personas['P6040'] > 18]

# Agrupar por 'id' y sumar los valores de 'P6210s1' para cada hogar
promedio_educacion_train = train_personas_mayores.groupby('id')['P6210s1'].sum().reset_index()
promedio_educacion_train.rename(columns={'P6210s1': 'Suma_P6210s1'}, inplace=True)

# Unir con el DataFrame de hogares en base al 'id'
train_hogares = train_hogares.merge(promedio_educacion_train, on='id', how='left')

# Calcular el promedio de educación por hogar
train_hogares['Promedio_Educacion'] = train_hogares['Suma_P6210s1'] / train_hogares['Nper']

# Repetir el proceso para el conjunto de prueba

# Filtrar personas mayores de 18 años en el dataset de prueba
test_personas_mayores = test_personas[test_personas['P6040'] > 18]

# Agrupar por 'id' y sumar los valores de 'P6210s1' para cada hogar
promedio_educacion_test = test_personas_mayores.groupby('id')['P6210s1'].sum().reset_index()
promedio_educacion_test.rename(columns={'P6210s1': 'Suma_P6210s1'}, inplace=True)

# Unir con el DataFrame de hogares en base al 'id'
test_hogares = test_hogares.merge(promedio_educacion_test, on='id', how='left')

# Calcular el promedio de educación por hogar
test_hogares['Promedio_Educacion'] = test_hogares['Suma_P6210s1'] / test_hogares['Nper']

In [7]:
# Crear una nueva variable que sea promedio de personas por habitacion

train_hogares['hacinamiento'] = train_hogares['Nper'] / train_hogares['P5010']

# Calcular el ratio de personas por dormitorio (Aglomeracion) en test_hogares
test_hogares['hacinamiento'] = test_hogares['Nper'] / test_hogares['P5010']

In [8]:
# Crear la variable `capacitacion_financiera` en el set de entrenamiento

capacitacion_train = train_personas.groupby('id')['P7510s5'].apply(lambda x: 1 if (x == 1).any() else 0).reset_index()
capacitacion_train.rename(columns={'P7510s5': 'capacitacion_financiera'}, inplace=True)

# Unir con el DataFrame de hogares en base al 'id'
train_hogares = train_hogares.merge(capacitacion_train, on='id', how='left')

# Crear la variable `capacitacion_financiera` en el set de prueba
capacitacion_test = test_personas.groupby('id')['P7510s5'].apply(lambda x: 1 if (x == 1).any() else 0).reset_index()
capacitacion_test.rename(columns={'P7510s5': 'capacitacion_financiera'}, inplace=True)

# Unir con el DataFrame de hogares en base al 'id'
test_hogares = test_hogares.merge(capacitacion_test, on='id', how='left')

In [9]:
# Crear la variable `jefe_sex` para el conjunto de entrenamiento
# Filtrar las personas que son jefes de hogar
jefes_train = train_personas[train_personas['P6050'] == 1][['id', 'P6020']]

# Renombrar la columna `P6020` a `jefe_sex`
jefes_train.rename(columns={'P6020': 'jefe_sex'}, inplace=True)

# Unir con el DataFrame de hogares en base al 'id'
train_hogares = train_hogares.merge(jefes_train, on='id', how='left')

# Crear la variable `jefe_sex` para el conjunto de prueba
jefes_test = test_personas[test_personas['P6050'] == 1][['id', 'P6020']]
jefes_test.rename(columns={'P6020': 'jefe_sex'}, inplace=True)
test_hogares = test_hogares.merge(jefes_test, on='id', how='left')

## Estadistica descriptiva

In [10]:
# Estadistica descriptiva de los datos (falta completar)

## Modelos
1. Regresion lineal por ingreso (Santi) 3
2. Logit con 0 y 1 (Luis) 3
3. Random Forest (Todos) 3
4. Boosting (Gina) 3

Trabajar con train_hogares (separando 70/30). test_hogares recien hay que volver a usarlo al final de todo para mandar nuestras predicciones

### 1. Regresion lineal

### 2. Logit

### 3. Random Forest

### 4. Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
# Definir los modelos
modelo_ada1 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=100)
modelo_ada2 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100)
modelo_ada3 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=150)


In [ ]:
# Dividir la base de datos en entrenamiento y testeo
train_df, test_df = train_test_split(train_hogares, test_size=0.3, random_state=123)

# Definir las variables predictoras (X) y la variable objetivo (y)
X_train = train_df[['hacinamiento', 'capacitacion_financiera', 'P5130', 'P5090', 'jefe_sex']]
y_train = train_df['Pobre']
X_test = test_df[['hacinamiento', 'capacitacion_financiera', 'P5130', 'P5090', 'jefe_sex']]
y_test = test_df['Pobre']

In [ ]:
# Entrenar los modelos
modelo_ada1.fit(X_train, y_train)
modelo_ada2.fit(X_train, y_train)
modelo_ada3.fit(X_train, y_train)

In [ ]:
# Calcular las probabilidades predichas para los tres modelos
test_df['prob_hat_ada1'] = modelo_ada1.predict_proba(X_test)[:, 1]
test_df['prob_hat_ada2'] = modelo_ada2.predict_proba(X_test)[:, 1]
test_df['prob_hat_ada3'] = modelo_ada3.predict_proba(X_test)[:, 1]


In [ ]:
# Graficar las curvas ROC para los tres modelos
plt.figure(figsize=(8, 6))
for i, (modelo, prob_col) in enumerate(zip(
        ['Modelo AdaBoost 1', 'Modelo AdaBoost 2', 'Modelo AdaBoost 3'], 
        ['prob_hat_ada1', 'prob_hat_ada2', 'prob_hat_ada3']
    )):
    fpr, tpr, _ = roc_curve(y_test, test_df[prob_col])
    auc = roc_auc_score(y_test, test_df[prob_col])
    plt.plot(fpr, tpr, label=f"{modelo} (AUC = {auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')  # Línea aleatoria
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curvas ROC')
plt.legend()
plt.show()


In [ ]:
#funcionamiento 
hogares_testeo = test_hogares[['hacinamiento', 'capacitacion_financiera', 'P5130', 'P5090', 'jefe_sex']]

# Calcular probabilidades predichas
test_hogares['Pobre_est_m1'] = modelo_ada1.predict_proba(hogares_testeo)[:, 1]
test_hogares['Pobre_est_m2'] = modelo_ada2.predict_proba(hogares_testeo)[:, 1]
test_hogares['Pobre_est_m3'] = modelo_ada3.predict_proba(hogares_testeo)[:, 1]

# Convertir probabilidades a predicciones binarias con un umbral
umbral = 0.5 #(usamos la demostracion de clase para justificar el umbral)
test_hogares['Pobre_m1_bin'] = (test_hogares['Pobre_est_m1'] >= umbral).astype(int)
test_hogares['Pobre_m2_bin'] = (test_hogares['Pobre_est_m2'] >= umbral).astype(int)
test_hogares['Pobre_m3_bin'] = (test_hogares['Pobre_est_m3'] >= umbral).astype(int)

In [ ]:
# Crear el DataFrame final
df_modelo1 = test_hogares[['id', 'Pobre_m1_bin']]
df_modelo2 = test_hogares[['id', 'Pobre_m2_bin']]
df_modelo3 = test_hogares[['id', 'Pobre_m3_bin']]

df_modelo1.to_csv('prediccion modelo 1.csv', index=False, encoding='utf-8')
df_modelo2.to_csv('prediccion modelo 2.csv', index=False, encoding='utf-8')
df_modelo3.to_csv('prediccion modelo 3.csv', index=False, encoding='utf-8')